In [49]:
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Gemini API key setup complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Gemini API key setup complete.


In [50]:
from google.adk.agents import Agent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search
from google.genai import types 

print("ADK components imported successfully.") 







ADK components imported successfully.


In [51]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]

    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
        </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("Helper functions defined.")

Helper functions defined.


In [52]:
retry_config=types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1, # Initial delay before first retry (in seconds)
    http_status_codes=[429, 500, 503, 504] # Retry on these HTTP errors
)

In [53]:
root_agent = Agent (
    name="prakriti_agent",
    model=Gemini(
    model_name="gemini-2.5-flash-lite",
    api_key=GOOGLE_API_KEY,
    retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure",
    tools=[google_search],
)

print("Root Agent defined.")

Root Agent defined.


In [54]:
runner = InMemoryRunner(agent=root_agent)

print(" Runner created.")

 Runner created.


In [55]:
response = await runner.run_debug(
    "What is the Agent Development Kit from Google? What languages is the SDK available in?"
)


 ### Created new session: debug_session_id

User > What is the Agent Development Kit from Google? What languages is the SDK available in?
prakriti_agent > The Agent Development Kit (ADK) from Google is a flexible, modular, and open-source framework designed for developing and deploying AI agents. It aims to simplify the process of building, deploying, and orchestrating agentic architectures, making it feel more like traditional software development. While optimized for Gemini and the Google ecosystem, ADK is model-agnostic and deployment-agnostic, allowing for compatibility with other frameworks.

Key features of the ADK include:
*   **Multi-Agent Architecture** It allows for building modular and scalable applications by composing multiple specialized agents.
*   **Rich Tool Ecosystem** Agents can be equipped with diverse capabilities using pre-built tools (like Search and Code Execution), custom functions, and integrations with third-party libraries.
*   **Flexible Orchestration** De

In [56]:
response = await runner.run_debug("What's the weather in London?")


 ### Continue session: debug_session_id

User > What's the weather in London?
prakriti_agent > The weather in London, UK, is currently mostly cloudy with a temperature of 57°F (14°C), feeling like 55°F (13°C). There is a 0% chance of rain, and the humidity is around 88%.

The forecast for today, Thursday, November 13, 2025, predicts cloudy conditions during the day and rain showers at night, with a 25% chance of rain during the day and a 50% chance at night. Temperatures are expected to range between 54°F (12°C) and 61°F (16°C), with humidity around 89%.


In [57]:
response = await runner.run_debug(" Who was the first Canadian President ?")


 ### Continue session: debug_session_id

User >  Who was the first Canadian President ?
prakriti_agent > Canada does not have a President, as it is a constitutional monarchy and a parliamentary democracy. The head of state in Canada is the reigning monarch of the United Kingdom (currently King Charles III), who is represented in Canada by the Governor General.

The head of government in Canada is the Prime Minister. The first Prime Minister of Canada was Sir John A. Macdonald, who served from 1867 to 1873 and again from 1878 to 1891.


In [62]:
!adk create sample-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY

Non-empty folder already exist: '/kaggle/working/sample-agent'
Override existing content? [y/N]: ^C
Aborted!


In [66]:
!ls -la /kaggle/working/sample-agent/ 

total 20
drwxr-xr-x 2 root root 4096 Nov 13 05:06 .
drwxr-xr-x 4 root root 4096 Nov 13 05:06 ..
-rw-r--r-- 1 root root  257 Nov 13 05:06 agent.py
-rw-r--r-- 1 root root   82 Nov 13 05:06 .env
-rw-r--r-- 1 root root   20 Nov 13 05:06 __init__.py


In [73]:
url_prefix = get_adk_proxy_url()
print(f"Open your ADK web UI at : {url_prefix}")

Open your ADK web UI at : /k/276992473/eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2IiwidHlwIjoiSldUIn0..o4vG67CxLuOhFqoKACTq7w.lT-bjsv433uHWZtnDfpSoKHPcrxw5T-KfS6u5gGCShMP5SgIZPqx20L_o5HpziOvOi_v2ku9QFDfxBH6hUb-drqcNVwW4anY4mN2TIK7BCbcXvNS_Ci7MOVep8GP6QG4kb80Id1FVx26jnXrMCa9r6y_IYuhuWBc8qn9VvZvEQ7hIQsxpxeGljxiDw9tUj4z-MuZXrNTF3F0T3fOP3MYqc23IbcSG4R8r6idpc5XYJE.YgLvpjZcIezc2i_SY6uLsg/proxy/proxy/8000


In [ ]:
!adk web --url_prefix {url_prefix}

/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [152]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http: